<a href="https://colab.research.google.com/github/gimah/Allan/blob/master/Share_to_Audience_Case_Study_Homeless.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Study - Homeless

In [ ]:
pip install altair

In [ ]:
pip install config

In [ ]:
import pandas as pd
from config import *

df = pd.read_csv('/content/homeless.csv')
df.head()

FileNotFoundError: ignored

Select only some fields

In [ ]:
df.Citizenship.unique()

In [ ]:
df['Territory'] = df['Territory'].str.replace('Trentino Alto Adige', 'Trentino-Alto Adige/Südtirol')

Focus on total age, total sex and total citizenship

In [ ]:
df_tot = df[(df['Age'] == 'TOTAL') & (df['Sex'] == 'T') &  (df['Citizenship'] == 'TOTAL')]

## Draw a preliminary map

In [ ]:
height=300
width=300
labelFontSize=15
titleFontSize=20 # axis title font size
fontSize=25 # title font size
textFontSize=22
offset=25

# level 3
color='#636466'
iColor='#80C11E'
color2 = ['#636466','#80C11E']
color6 = ['#636466','#80C11E', '#D8519E', '#E37A45','#773B9A','#E6CB03']
iColor2 = ['#80C11E','#636466']


def configure_layout(chart):
    return chart.configure_axis(
        labelFontSize=labelFontSize,
        titleFontSize=titleFontSize
    ).configure_title(fontSize=fontSize,offset=offset
    ).configure_legend(titleFontSize=titleFontSize, labelFontSize=labelFontSize)

In [ ]:
import altair as alt

# Load the TopoJSON file by URL
url = "https://raw.githubusercontent.com/openpolis/geojson-italy/master/topojson/limits_IT_regions.topo.json"


source = alt.topo_feature(url, "regions")

chart = alt.Chart(source).mark_geoshape().encode(
    tooltip='properties.reg_name:N',
    color=alt.Color('Value:Q')
).project('mercator').properties(
    width=500,
    height=500
).transform_lookup(
    lookup='properties.reg_name',
    from_=alt.LookupData(df_tot, 'Territory', ['Territory', 'Value'])
).properties(title='Homeless in Italy in 2021')

configure_layout(chart)

# Focus on the key message

In [ ]:
df_tot['Territory'] = df_tot['Territory'].str.replace('Trentino-Alto Adige/Südtirol','Trentino Alto Adige')

chart = alt.Chart(df_tot).mark_bar(color=color).encode(
    y = alt.Y('Territory:N', sort='-x', axis=alt.Axis(title='')),
    x = alt.X('Value:Q', axis=alt.Axis(title='Number of homeless')),

).properties(title='Homeless in Italy in 2021')

configure_layout(chart)

In [ ]:
chart

In [ ]:
chart.save('chart.html')

Normalize by population

In [ ]:
population = pd.read_csv('/content/population.csv')
population.head()

In [ ]:
pop_tot = population[(population['Age'] == 'TOTAL') & (population['Sex'] == 'T')]

In [ ]:
len(df_tot), len(pop_tot)

In [ ]:
df_tot.columns

In [ ]:
df_tot.Territory.unique()

In [ ]:
pop_tot.Territory.unique()

Join the two datasets

In [ ]:
df_tot = df_tot[['Value', 'ITTER107']]
pop_tot = pop_tot[['Value', 'ITTER107','Territory']]

In [ ]:
df3 = pop_tot.set_index('ITTER107').join(df_tot.set_index('ITTER107'),lsuffix='_pop', rsuffix='_hom').reset_index()

Calculate ratio: number of homeless people over 1000 inhabitants

In [ ]:
df3['Ratio'] = df3['Value_hom']/df3['Value_pop']*1000

In [ ]:
chart = alt.Chart(df3).mark_bar().encode(
    y = alt.Y('Territory', sort='-x', axis=alt.Axis(title='')),
    x = alt.X('Ratio', axis=alt.Axis(tickCount=4,title='')),
    color=alt.condition(alt.datum.Ratio > 2, alt.value(iColor), alt.value('lightgray'))

).properties(width=500,title='Number of homeless people in a population of 1,000')

configure_layout(chart)

# Adding a context

In [ ]:
chart = alt.Chart(df3).mark_bar().encode(
    y = alt.Y('Territory', sort='-x', axis=alt.Axis(title='')),
    x = alt.X('Ratio', axis=alt.Axis(tickCount=4,title='')),
    color=alt.condition(alt.datum.Ratio > 2, alt.value(iColor), alt.value('lightgray'))

).properties(width=500,title=alt.TitleParams(
    text=["Together, Let's Make a Difference:","Support Our Project to Help the Homeless!"],
    subtitle=['Homelessness is a heartbreaking reality that leaves individuals and families without a stable home,','leading to devastating consequences such as poor health and social isolation.'],
    subtitleFontSize=18
))


configure_layout(chart)

In [ ]:

# Create an Altair chart with an image mark
image1 = alt.Chart(pd.DataFrame({'image_url': ['source/homeless1.png']})).mark_image(
    width=200,
    height=200,
).encode(
    url='image_url',
    x=alt.value(0),  # pixels from left
    y=alt.value(50)
)

image2 = alt.Chart(pd.DataFrame({'image_url': ['source/homeless2.png']})).mark_image(
    width=200,
    height=200,
).encode(
    url='image_url',
    x=alt.value(0),  # pixels from left
    y=alt.value(300)
)

configure_layout(image1 + image2| chart)

# Adding next steps

In [ ]:
ns = pd.read_csv('/content/next_steps.csv')
ns.head()

In [ ]:
donuts = None
for index, row in ns.iterrows():
    # Create a pie chart for the current row
    curr_ns = pd.DataFrame(
        {'Category': ['A', 'B'],
         'Value': [row['Allocation'], 100-row['Allocation']]
        }
    )
    donut = alt.Chart(curr_ns).mark_arc(outerRadius=30,innerRadius=20).encode(
        theta=alt.Theta("Value:Q",stack=True),
        color=alt.Color("Category:N",scale=alt.Scale(range=[iColor, 'lightgray']),legend=None)
    )
    title = alt.Chart(curr_ns).mark_text(text=row['Category'], y=0, size=16)
    text = alt.Chart(curr_ns).mark_text(text=f"{row['Allocation']}%", color=iColor, size=16)
    donut = donut.properties(
        height=100,
        width=100
    )
    if index == 0:
        donuts = title+donut+text
    else:
        donuts = alt.hconcat(donuts, title+donut+text)
donuts = donuts.properties(title='Our visionary plan to harness the funds')
configure_layout(donuts)

In [ ]:
configure_layout(alt.vconcat(image1 + image2| chart,donuts))